In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,  BatchNormalization, GlobalAveragePooling2D,  Input
import torch.optim.lr_scheduler as lr_scheduler


In [ ]:
!mkdir -p ~/.kaggle
!cp /kaggle/input/kagglejson/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset -p /kaggle/working


In [ ]:
%%capture
!unzip /kaggle/working/new-plant-diseases-dataset.zip -d /kaggle/working


## Data Preprocessing

In [ ]:

def check_image_dimensions(folder_path):
    """
    Checks if all images in a folder and its subfolders have the same dimensions.

    :param folder_path: Path to the main folder
    :return: True if all images have the same dimensions, otherwise False and the inconsistent dimensions
    """
    dimensions = None
    inconsistent_files = []
    total_files = sum(len(files) for _, _, files in os.walk(folder_path))  # Count all files

    # Traverse the folder and its subfolders
    with tqdm(total=total_files, desc="Processing images", unit="file") as pbar:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    # Open the image file
                    with Image.open(file_path) as img:
                        img_dimensions = img.size  # Returns (width, height)
                        if dimensions is None:
                            # Set the reference dimensions
                            dimensions = img_dimensions
                        elif img_dimensions != dimensions:
                            # Add to the list of inconsistent files if dimensions don't match
                            inconsistent_files.append((file_path, img_dimensions))
                except Exception as e:
                    # Handle errors when processing non-image files or corrupted images
                    print(f"Error processing {file_path}: {e}")
                finally:
                    pbar.update(1)  # Update the progress bar

    if inconsistent_files:
        # Print all files with inconsistent dimensions
        print("The following images have inconsistent dimensions:")
        for file_path, dim in inconsistent_files:
            print(f"{file_path} - Dimensions: {dim}")
        return False
    else:
        # Print success message if all dimensions are consistent
        print("All images have the same dimensions:", dimensions)
        return True

# Specify the main folder to check
folder_path = "/kaggle/working/"
check_image_dimensions(folder_path)


In [ ]:

# Configuration of the image data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalization: scales pixel values between 0 and 1
    rotation_range=20,  # Random rotation up to 20 degrees
    width_shift_range=0.2,  # Random horizontal shift up to 20% of the width
    height_shift_range=0.2,  # Random vertical shift up to 20% of the height
    zoom_range=0.2,  # Random zoom up to 20%
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'  # Filling missing pixels with the nearest value
)

# Generator for the validation or test set (without augmentation)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)  # Only normalization

# Load images from the file system
train_generator = train_datagen.flow_from_directory(
    "/kaggle/working/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",  # Path to the training dataset
    target_size=(256, 256),  # Resize images to 224x224
    batch_size=32,  # Number of images per batch
    class_mode='categorical' , # Type of labels: 'categorical' for multi-class classification
    shuffle=True
)

valid_generator = validation_datagen.flow_from_directory(
    "/kaggle/working/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Visualization of 5 images before and after processing
def plot_images_before_after(generator, num_images=5):
    # Get a batch of images from the generator
    images, _ = next(generator)

    # Number of images to display
    num_images = min(num_images, len(images))

    fig, axes = plt.subplots(num_images, 2, figsize=(10, num_images * 3))
    fig.suptitle('Original vs Augmented Images', fontsize=16)

    for i in range(num_images):
        # Show the original image
        axes[i, 0].imshow(images[i])
        axes[i, 0].set_title("Original Image")
        axes[i, 0].axis('off')
        print("Before:",images[i])

        # Create a new augmented image (simulate data augmentation)
        augmented_img = train_datagen.random_transform(images[i])
        axes[i, 1].imshow(augmented_img)
        axes[i, 1].set_title("Augmented Image")
        axes[i, 1].axis('off')
        print("After:",augmented_img)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# Get a batch from the validation generator to visualize the original images
valid_generator.reset()  # Ensure the generator starts from the beginning
images, labels = next(valid_generator)

# Denormalize images for correct visualization
#images = images * 255.0

# Display 5 images before and after processing
plot_images_before_after(valid_generator, num_images=5)

print("Dictionary containing classes:")
print(train_generator.class_indices)

print("List of classes with index linked to dataset")
print(train_generator.classes)

In [ ]:
# Import necessary libraries
import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,  BatchNormalization, GlobalAveragePooling2D



import matplotlib.pyplot as plt


## Define Model Architectures

### Baseline CNN Model

In [ ]:
def base_cnn_model(input_shape=(256, 256, 3), num_classes=38):

    """

    Custom CNN model for plant disease detection.

    :param input_shape: Tuple representing the shape of input images.

    :param num_classes: Integer, number of output classes.

    :return: Compiled Keras model.

    """

    model = Sequential()



    # Convolutional Block 1

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))

    model.add(MaxPooling2D(pool_size=(2, 2)))



    # Convolutional Block 2

    model.add(Conv2D(64, (3, 3), activation='relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))



    # Convolutional Block 3

    model.add(Conv2D(128, (3, 3), activation='relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))



    # Fully Connected Layers

    model.add(Flatten())

    model.add(Dense(256, activation='relu'))

    model.add(Dropout(0.5))  # Dropout for regularization

    model.add(Dense(num_classes, activation='softmax'))  # Softmax for multi-class classification





    return model


In [ ]:
# Create the CNN model

model = base_cnn_model(input_shape=(256, 256, 3), num_classes=38)



# Display the model architecture

model.summary()


In [ ]:
def visualize_model_architecture():

    layers = [

        "\nInput (256x256x3)",

        "Conv2D (32 filters)",

        "MaxPooling2D",

        "Conv2D (64 filters)",

        "MaxPooling2D",

        "Conv2D (128 filters)",

        "MaxPooling2D",

        "Flatten",

        "Dense (256 units)",

        "Dropout",

        "Dense (38 classes)"

    ]

    

    # Create a simple bar plot

    plt.figure(figsize=(14, 10))

    for i, layer in enumerate(layers):

        plt.text(0.5, len(layers) - i - 0.5, layer, fontsize=12, ha="center", va="center")

        plt.hlines(y=len(layers) - i - 1, xmin=0.2, xmax=0.8, color="black")

    

    plt.xticks([])

    plt.yticks([])

    plt.title("Model Architecture\n")

    plt.show()



visualize_model_architecture()


### Improved CNN Model

### Improvements to the Model



To improve the architecture:



1. **Add More Convolutional Layers**  

   Increase the model's depth to extract more complex features.



2. **Use Batch Normalization**  

   Normalize activations within layers to stabilize and speed up training.



3. **Adjust Dropout and Fully Connected Layers**  

   Add more dense layers with dropouts for better generalization.



4. **Global Average Pooling**  

   Replace the `Flatten()` layer with `GlobalAveragePooling2D` for more efficient feature aggregation.



5. **Learning Rate Scheduler**  

   Include learning rate adjustment for smoother training.


In [ ]:


def improved_cnn_model(input_shape=(256, 256, 3), num_classes=38):

    """

    Improved CNN model for plant disease detection.

    :param input_shape: Tuple representing the shape of input images.

    :param num_classes: Integer, number of output classes.

    :return: Compiled Keras model.

    """

    model = Sequential()



    # Convolutional Block 1

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))

    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))



    # Convolutional Block 2

    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))



    # Convolutional Block 3

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))



    # Convolutional Block 4

    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))

    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))



    # Global Average Pooling instead of Flatten

    model.add(GlobalAveragePooling2D())



    # Fully Connected Layers

    model.add(Dense(512, activation='relu'))

    model.add(Dropout(0.5))

    model.add(Dense(256, activation='relu'))

    model.add(Dropout(0.3))



    # Output Layer

    model.add(Dense(num_classes, activation='softmax'))



    return model


In [ ]:
# Create the improved CNN model

improved_model = improved_cnn_model(input_shape=(256, 256, 3), num_classes=38)



# Display the improved model architecture

improved_model.summary()

In [ ]:

def visualize_improved_model_architecture():

    layers = [

        "\nInput (256x256x3)",

        "Conv2D (32 filters) + BatchNorm",

        "MaxPooling2D",

        "Conv2D (64 filters) + BatchNorm",

        "MaxPooling2D",

        "Conv2D (128 filters) + BatchNorm",

        "MaxPooling2D",

        "Conv2D (256 filters) + BatchNorm",

        "MaxPooling2D",

        "GlobalAveragePooling2D",

        "Dense (512 units) + Dropout",

        "Dense (256 units) + Dropout",

        "Dense (38 classes)"

    ]

    

    plt.figure(figsize=(14, 10))

    for i, layer in enumerate(layers):

        plt.text(0.5, len(layers) - i - 0.5, layer, fontsize=12, ha="center", va="center")

        plt.hlines(y=len(layers) - i - 1, xmin=0.2, xmax=0.8, color="black")

    

    plt.xticks([])

    plt.yticks([])

    plt.title("Improved Model Architecture\n")

    plt.show()



visualize_improved_model_architecture()


### MobileNet Model

In [ ]:

def mobilenet_model(input_shape=(256, 256, 3), num_classes=38):


    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)


    base_model.trainable = False

  
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False) 
    x = GlobalAveragePooling2D()(x) 
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)  
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x) 
    outputs = Dense(num_classes, activation='softmax')(x) 

    
    model = Model(inputs, outputs)

    return model


In [ ]:
mobileNetModel = mobilenet_model()

## Training Process

In [ ]:
# Define the schedule
initial_learning_rate = 1e-5
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=10000,  # Number of steps before decay
    decay_rate=0.96,    # Decay factor
    staircase=True      # Apply decay in discrete intervals
)

# Compile the model with the schedule
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
               metrics=[
                'accuracy',  # Default accuracy
                tf.keras.metrics.Precision(name='precision'),  # Precision metric
                tf.keras.metrics.Recall(name='recall'),        # Recall metric
                tf.keras.metrics.AUC(name='auc'),             # AUC metric
            ])

improved_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
               metrics=[
                'accuracy',  # Default accuracy
                tf.keras.metrics.Precision(name='precision'),  # Precision metric
                tf.keras.metrics.Recall(name='recall'),        # Recall metric
                tf.keras.metrics.AUC(name='auc'),             # AUC metric
            ])

mobileNetModel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
               metrics=[
                'accuracy',  # Default accuracy
                tf.keras.metrics.Precision(name='precision'),  # Precision metric
                tf.keras.metrics.Recall(name='recall'),        # Recall metric
                tf.keras.metrics.AUC(name='auc'),             # AUC metric
            ])

In [ ]:
history1 = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=40,
    batch_size=128
)

In [ ]:
history2 = improved_model.fit(
    train_generator,  
    validation_data=valid_generator,
    epochs=40,
    batch_size=128
)

In [ ]:
history3 = mobileNetModel.fit(
    train_generator,  
    validation_data=valid_generator,
    epochs=40,
    batch_size=128
)

### Save Model

In [ ]:
# Save the final model after training
model.save('model.h5')

# Optionally, save the history for later analysis
import pickle

with open('history.pkl', 'wb') as file:
    pickle.dump(history1.history, file)

### Plot Model

In [ ]:
import pickle
import matplotlib.pyplot as plt

# Plot the combined history
def plot_training_history(history):
    acc = history['accuracy']
    val_acc = history['val_accuracy']
    loss = history['loss']
    val_loss = history['val_loss']
    epochs = range(1, len(acc) + 1)

    # Plot accuracy
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the plotting function
plot_training_history(history1)


## Testing and Evaluation

In [ ]:
def check_image_dimensions(folder_path):
    """
    Checks if all images in a folder and its subfolders have the same dimensions.

    :param folder_path: Path to the main folder
    :return: True if all images have the same dimensions, otherwise False and the inconsistent dimensions
    """
    dimensions = None
    inconsistent_files = []
    total_files = sum(len(files) for _, _, files in os.walk(folder_path))  # Count all files

    # Traverse the folder and its subfolders
    with tqdm(total=total_files, desc="Processing images", unit="file") as pbar:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    # Open the image file
                    with Image.open(file_path) as img:
                        img_dimensions = img.size  # Returns (width, height)
                        if dimensions is None:
                            # Set the reference dimensions
                            dimensions = img_dimensions
                        elif img_dimensions != dimensions:
                            # Add to the list of inconsistent files if dimensions don't match
                            idnconsistent_files.append((file_path, img_dimensions))
                except Exception as e:
                    # Handle errors when processing non-image files or corrupted images
                    print(f"Error processing {file_path}: {e}")
                finally:
                    pbar.update(1)  # Update the progress bar

    if inconsistent_files:
        # Print all files with inconsistent dimensions
        print("The following images have inconsistent dimensions:")
        for file_path, dim in inconsistent_files:
            print(f"{file_path} - Dimensions: {dim}")
        return False
    else:
        # Print success message if all dimensions are consistent
        print("All images have the same dimensions:", dimensions)
        return True

# Specify the main folder to check
folder_path = "/kaggle/working/"
check_image_dimensions(folder_path)


In [ ]:

from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Function to load and evaluate a model
def load_and_evaluate_model(model_path, test_generator):
    # Load the model
    model = load_model(model_path)
    print(f"Evaluando modelo: {model_path}")

    # Make predictions
    test_generator.reset()
    predictions = model.predict(test_generator, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes  # Get the real labels
    class_labels = list(test_generator.class_indices.keys())  # Class labels

    return predicted_classes, true_classes, class_labels


In [ ]:

# Setting up the generator for raw images
test_generator = validation_datagen.flow_from_directory(
    directory="/kaggle/working/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid",
    target_size=(256, 256),
    batch_size=32,
    class_mode=None,
    shuffle=False
)


In [ ]:
# # Evaluate the Model 4
predicted_classes4, true_classes4, class_labels4 = load_and_evaluate_model(
    'baselinemodel.h5', test_generator
)

# # Evaluate the Model 5
predicted_classes5, true_classes5, class_labels5 = load_and_evaluate_model(
    'improvedmodel.h5', test_generator
)


In [ ]:
#CLASSIFICATION REPORT

from sklearn.metrics import classification_report

def print_classification_report(model_name, true_classes, predicted_classes, class_labels):
    # Print the classification report with model name
    print(f"\nClassification Report for {model_name}:\n")
    report = classification_report(true_classes, predicted_classes, target_names=class_labels)
    print(report)


# # Print the Classification Report for Model 1
# print_classification_report("Model 1", true_classes1, predicted_classes1, class_labels1)

# Print the Classification Report for Model 2
# print_classification_report("Model 2", true_classes2, predicted_classes2, class_labels2)

# # Print the Classification Report for Model 4
print_classification_report("Model 4", true_classes4, predicted_classes4, class_labels4)

# # Print the Classification Report for Model 5
print_classification_report("Model 5", true_classes5, predicted_classes5, class_labels5)


In [ ]:

#CONFUSION MATRIX

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_and_metrics(true_classes, predicted_classes, class_labels, title="Confusion Matrix"):
    # Calculate the confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes, labels=range(len(class_labels)))

    # Adjust figure size to avoid clutter
    fig, ax = plt.subplots(figsize=(12, 10))  # Larger figure size for more space

    # Display the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
    disp.plot(cmap=plt.cm.Blues, ax=ax, colorbar=True)

    # Adjust label rotation and size
    ax.set_xticklabels(class_labels, rotation=90, fontsize=10, ha='center')  # Vertical labels
    ax.set_yticklabels(class_labels, fontsize=10)

    # Adjust text size and spacing inside the matrix
    for text in ax.texts:
        text.set_fontsize(8)       # Adjust font size
        text.set_ha('center')     # Center align
        text.set_va('center')     # Vertical align
        text.set_position((text.get_position()[0], text.get_position()[1] - 0.2))  # Adjust spacing

    # Add title to the plot
    plt.title(title, fontsize=14)
    plt.tight_layout()
    plt.show()

# Plot results for Model 1
# plot_confusion_and_metrics(true_classes1, predicted_classes1, class_labels1, title="Confusion Matrix - Model 1")

# # Plot results for Model 2
# plot_confusion_and_metrics(true_classes2, predicted_classes2, class_labels2, title="Confusion Matrix - Model 2")

# # Plot results for Model 4
plot_confusion_and_metrics(true_classes4, predicted_classes4, class_labels4, title="Confusion Matrix - Model 4")

# Plot results for Model 5
plot_confusion_and_metrics(true_classes5, predicted_classes5, class_labels5, title="Confusion Matrix - Model 5")


In [ ]:

# ROC and AUC Curves

from tensorflow.keras.models import load_model
from sklearn.preprocessing import label_binarize

def get_predicted_probabilities(model_path, test_generator):
    # Load the model
    model = load_model(model_path)

    # Reset the generator and get the predicted probabilities
    test_generator.reset()
    predicted_probabilities = model.predict(test_generator, verbose=1)
    return predicted_probabilities

# Obtaining the probabilities for Model 1
# predicted_probabilities1 = get_predicted_probabilities('/content/final_model20+20_128_1e-5.h5', test_generator)

# Obtaining the probabilities for Model 2
# predicted_probabilities2 = get_predicted_probabilities('final_improved_model10+10,128.h5', test_generator)

# Obtaining the probabilities for Model 4
predicted_probabilities4 = get_predicted_probabilities('baselinemodel.h5', test_generator)

# Obtaining the probabilities for Model 5
predicted_probabilities5 = get_predicted_probabilities('improvedmodel.h5', test_generator)


In [ ]:

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

def plot_roc_curve(true_classes, predicted_probabilities, class_labels, model_name):
    # Convert true classes to binary format (One-Hot Encoding)
    true_classes_bin = label_binarize(true_classes, classes=range(len(class_labels)))

    # Initialize the figure for the graph
    plt.figure(figsize=(12, 8))

    # Plot the ROC curve for each class
    for i in range(len(class_labels)):
        fpr, tpr, thresholds = roc_curve(true_classes_bin[:, i], predicted_probabilities[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'Class {class_labels[i]} (AUC = {roc_auc:.2f})')

    # Random ROC curve (diagonal line)
    plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier (AUC = 0.5)')

    # Final adjustments to the chart
    plt.title(f'ROC curves for {model_name}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)

    # Place the legend outside the graph to avoid overlaps
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fancybox=True, shadow=True, ncol=1)

    # Adjust the layout so that the labels do not overlap
    plt.tight_layout()
    plt.show()

#  Plot the ROC and AUC Curves for Model 1
# plot_roc_curve(true_classes1, predicted_probabilities1, class_labels1, model_name="Model 1")

# # Plot the ROC and AUC Curves for Model 2
# plot_roc_curve(true_classes2, predicted_probabilities1, class_labels2, model_name="Model 2")

# Plot the ROC and AUC Curves for Model 4
plot_roc_curve(true_classes4, predicted_probabilities4, class_labels4, model_name="Model 4")

# Plot the ROC and AUC Curves for Model 5
plot_roc_curve(true_classes5, predicted_probabilities5, class_labels5, model_name="Model 5")



In [ ]:


from sklearn.metrics import roc_auc_score
import numpy as np

def calculate_overall_auc(true_classes, predicted_probabilities, class_labels):
    # Convert true classes to binary format (One-Hot Encoding)
    true_classes_bin = label_binarize(true_classes, classes=range(len(class_labels)))

    # Calculate AUC for each class
    auc_per_class = []
    for i in range(len(class_labels)):
        roc_auc = roc_auc_score(true_classes_bin[:, i], predicted_probabilities[:, i])
        auc_per_class.append(roc_auc)

    # Calculate the average AUC
    overall_auc = np.mean(auc_per_class)
    print(f"AUC per class: {auc_per_class}")
    print(f"Overall AUC (Average): {overall_auc:.2f}")
    return overall_auc

# Example for Model 1
overall_auc1 = calculate_overall_auc(true_classes4, predicted_probabilities4, class_labels4)
# overall_auc2 = calculate_overall_auc(true_classes5, predicted_probabilities5, class_labels5)

### MobileNet needs Tensoflow version 2.10.0

In [ ]:
!pip install tensorflow==2.10.0

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from tqdm import tqdm

In [ ]:

# Evaluate the Model 3
predicted_classes3, true_classes3, class_labels3 = load_and_evaluate_model(
    'mobileNet20+20,128,1e-5.h5', test_generator
)


In [ ]:

# Print the Classification Report for Model 3
print_classification_report("Model 3", true_classes3, predicted_classes3, class_labels3)

In [ ]:
# Plot results for Model 3
plot_confusion_and_metrics(true_classes3, predicted_classes3, class_labels3, title="Confusion Matrix - Model 3")

In [ ]:

# Obtaining the probabilities for Model 3
predicted_probabilities3 = get_predicted_probabilities('/content/mobileNet20+20,128,1e-5.h5', test_generator)


In [ ]:
plot_roc_curve(true_classes3, predicted_probabilities3, class_labels3, model_name="Model 3")

In [ ]:
# Function to calculate the overall AUC
def calculate_overall_auc(true_classes, predicted_probabilities, class_labels):
    true_classes_bin = label_binarize(true_classes, classes=range(len(class_labels)))
    auc_values = [auc(*roc_curve(true_classes_bin[:, i], predicted_probabilities[:, i])[:2]) for i in range(len(class_labels))]
    overall_auc = np.mean(auc_values)
    print(f"Overall AUC: {overall_auc:.2f}")
    return overall_auc

overall_auc3 = calculate_overall_auc(true_classes3, predicted_probabilities3, class_labels3)